<a href="https://colab.research.google.com/github/ATOMconsortium/AMPL/blob/Tutorials/atomsci/ddm/examples/tutorials/11_CHEMBL26_SCN5A_IC50_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Building a Graph Convolutional Network Model for Drug Response Prediction</h1>

The ATOM Modeling PipeLine (AMPL; https://github.com/ATOMconsortium/AMPL) is an open-source, modular, extensible software pipeline for building and sharing models to advance in silico drug discovery.


## Time to run: 6 minutes

In [ ]:
!date

Thu May 20 22:57:16 UTC 2021


## Change your runtime to GPU 

Go to **Runtime** --> Change **runtime type** to "GPU"


## Goal: Use AMPL for predicting binding affinities -IC50 values- of ligands that could bind to human **Sodium channel protein type 5 subunit alpha** protein using Graph Convolutional Network Model. ChEMBL database is the source of the binding affinities (pIC50).

In this notebook, we describe the following steps using AMPL:

1.   Read a ML ready dataset
2.   Fit a Graph Convolutional model
3.   Predict pIC50 values of withheld compounds

## Set up
We first import the AMPL modules for use in this notebook.

The relevant AMPL modules for this example are listed below:

|module|Description|
|-|-|
|`atomsci.ddm.pipeline.model_pipeline`|The model pipeline module is used to fit models and load models for prediction.|
|`atomsci.ddm.pipeline.parameter_parser`|The parameter parser reads through pipeline options for the model pipeline.|
|`atomsci.ddm.utils.curate_data`|The curate data module is used for data loading and pre-processing.|
|`atomsci.ddm.utils.struct_utils`|The structure utilities module is used to process loaded structures.|


import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

## Install AMPL 

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

!pip install deepchem-nightly

import deepchem
deepchem.__version__

! pip install umap
! pip install llvmlite==0.34.0  --ignore-installed
! pip install umap-learn
! pip install molvs
! pip install bravado

In [ ]:
import deepchem as dc

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

In [ ]:
dc.__version__

'2.3.0'

In [ ]:
# We temporarily disable warnings for demonstration.
# FutureWarnings and DeprecationWarnings are present from some of the AMPL 
# dependency modules.
import warnings
warnings.filterwarnings('ignore')
import json
import requests
import sys

In [ ]:
import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

In [ ]:
import os
os.mkdir('chembl_activity_models')

**Let us display the dataset**

In [ ]:
import pandas as pd
import requests
import io
url = 'https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/datasets/ChEMBL26_SCN5A_IC50_human_ml_ready.csv'
download = requests.get(url).content
df = pd.read_csv(url, index_col=0)
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.iloc[0:5, 0:5]
df.to_csv('ChEMBL26_SCN5A_IC50_human_ml_ready.csv', index=False)

In [ ]:
df

,base_rdkit_smiles,compound_id,pIC50,relation,active
0,O=S(=O)(Nc1nccs1)c1ccc2c(c1)OCCN2c1ccc(Cl)cc1O...,CHEMBL3948159,5.806875,NaN,1
1,Fc1cccc(Cn2ccc3c(OC4CCN(Cc5ccccn5)CC4)ncnc32)c1F,CHEMBL2012299,7.698970,NaN,1
2,COc1cc(-c2cccc(F)c2)c(Cl)cc1-c1ncc(O)c2cc(S(=O...,CHEMBL4089326,5.000000,<,0
3,O=C(Nc1cccc(C(F)(F)F)c1-c1ccccn1)c1cccc(-c2ncc...,CHEMBL4084926,5.721246,NaN,1
4,CC(C)COc1ncc(-c2nn(C)c3cc(C(=O)NS(C)(=O)=O)ccc...,CHEMBL4280298,5.186552,NaN,1
...,...,...,...,...,...
1724,O=C(NCc1ccc(C(F)(F)F)cc1)C1c2ccccc2C(=O)N1CCc1...,CHEMBL2164387,5.795880,NaN,1
1725,COc1ccc(Cl)c(-c2ccc(NC(=O)c3ccnn3C)nc2N)c1,CHEMBL3589904,4.431798,NaN,1
1726,N#Cc1cc(F)ccc1N1CCOc2cc(S(=O)(=O)Nc3nccs3)ccc21,CHEMBL3955931,5.000000,<,0
1727,Clc1cn(Cc2ccccc2)c2ncnc(OC3CCN(Cc4cscn4)CC3)c12,CHEMBL2012181,7.096910,NaN,1


base_splitter
Description:	Type of splitter to use for train/validation split if temporal split used for test set. May be random, scaffold, or ave_min. The allowable choices are set in splitter.py

Check here for details, https://github.com/ATOMconsortium/AMPL/blob/master/atomsci/ddm/docs/PARAMETERS.md

In [ ]:
split_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"split_only": "True",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "False",
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [ ]:
split_params = parse.wrapper(split_config)

In [ ]:
split_model = mp.ModelPipeline(split_params)

In [ ]:
split_uuid = split_model.split_dataset()
split_uuid

number of features: 75


2021-05-20 23:00:28,712 Splitting data by scaffold
2021-05-20 23:00:42,179 Dataset split table saved to /content/ChEMBL26_SCN5A_IC50_human_ml_ready_train_valid_test_scaffold_4e9ac13a-8e48-43e6-9ca4-64fb9c571661.csv


'4e9ac13a-8e48-43e6-9ca4-64fb9c571661'

In [ ]:
!pip install --upgrade gspread

In [ ]:
!date

Thu May 20 23:00:45 UTC 2021


## Train the mode (~ 10 minutes)

In [ ]:
train_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"uncertainty": "False",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "True",
"split_uuid": "{}".format(split_uuid),
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [ ]:
train_params = parse.wrapper(train_config)

In [ ]:
train_model = mp.ModelPipeline(train_params)

## Train_model took ~ 18 minutes on a GPU  (~ 30 minutes on a CPU)

In [ ]:
train_model.train_model()

number of features: 75


2021-05-20 23:01:05,121 Previous dataset split restored
2021-05-20 23:01:59,971 Wrote model metadata to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/0b91d901-e1f2-49aa-b62a-a3676d90b3b1/model_metadata.json
2021-05-20 23:02:00,106 Wrote model metrics to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/0b91d901-e1f2-49aa-b62a-a3676d90b3b1/model_metrics.json


Wrote model tarball to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready_model_0b91d901-e1f2-49aa-b62a-a3676d90b3b1.tar.gz


In [ ]:
perf_df = cmp.get_filesystem_perf_results('/content/chembl_activity_models', pred_type='regression')
perf_df

Found data for 1 models under /content/chembl_activity_models


,model_uuid,ampl_version,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,learning_rate,dropouts,layer_sizes,best_epoch,max_epochs,rf_estimators,rf_max_features,rf_max_depth,xgb_gamma,xgb_learning_rate,model_choice_score,train_r2_score,train_rms_score,train_mae_score,train_num_compounds,valid_r2_score,valid_rms_score,valid_mae_score,valid_num_compounds,test_r2_score,test_rms_score,test_mae_score,test_num_compounds
0,0b91d901-e1f2-49aa-b62a-a3676d90b3b1,1.2.0,NN,/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv,graphconv,scaffold,r2,normalization,0.0007,"0.00,0.00,0.00","64,64,32",24,100,NaN,NaN,NaN,NaN,NaN,0.132874,0.738705,0.327762,0.239703,1210,0.132874,0.547236,0.406501,259,0.048182,0.684815,0.457751,260


In [ ]:
!date

Thu May 20 23:02:00 UTC 2021
